##### Step 1: Creating pose detection using MediaPipe Pose Landmarks

Reference: 
- https://colab.research.google.com/github/googlesamples/mediapipe/blob/main/examples/pose_landmarker/python/%5BMediaPipe_Python_Tasks%5D_Pose_Landmarker.ipynb
- https://ai.google.dev/edge/mediapipe/solutions/vision/pose_landmarker

In [1]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [2]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2

base_options = python.BaseOptions(model_asset_path='pose_detection_mediapipe/pose_landmarker_heavy.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

image = mp.Image.create_from_file("Dataset/full-lunges/4.jpg")

detection_result = detector.detect(image)

annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
cv2.imshow("Show image", cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()

##### Step 2: Calculating the angle between the ankle, knee, hip, and shoulders
These are the angles that we will be calculating for detecting the lunges

In [5]:
import math

def calculate_angle(landmark1, landmark2, landmark3):
    x1, y1, _ = landmark1.x, landmark1.y, landmark1.z
    x2, y2, _ = landmark2.x, landmark2.y, landmark2.z
    x3, y3, _ = landmark3.x, landmark3.y, landmark3.z

    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))

    if angle < 0:
        angle += 360

    if angle > 180:
        angle = 360 - angle

    return angle

In [3]:
from mediapipe import solutions

mp_pose = solutions.pose
def classify_pose(landmarks):
    label = "Unknown Pose"
    left_knee_angle = calculate_angle(landmarks[mp_pose.PoseLandmark.LEFT_HIP], 
                                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE], 
                                      landmarks[mp_pose.PoseLandmark.LEFT_ANKLE])
    
    right_knee_angle = calculate_angle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP], 
                                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE], 
                                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE])

    left_waist_angle = calculate_angle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER], 
                                    landmarks[mp_pose.PoseLandmark.LEFT_HIP], 
                                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE])
    
    right_waist_angle = calculate_angle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_KNEE])
    
    if (left_knee_angle > 75 and left_knee_angle < 115) and (right_knee_angle > 75 and right_knee_angle < 115):
        if (left_waist_angle > 160 and left_waist_angle < 180) or (right_waist_angle > 160 and right_waist_angle < 180):
            label = "Full Lunge"
        else:
            label = "Unknown Pose"
    elif (left_knee_angle > 120 and left_knee_angle < 180) and (right_knee_angle >= 115 and right_knee_angle < 145):
        # Right knee is bent
        label = "Half Lunge (R)"
    if (left_knee_angle >= 115 and left_knee_angle < 145) and (right_knee_angle > 120 and left_knee_angle < 180):
        # Left knee is bent
        label = "Half Lunge (L)"
    return {
        "label": label,
        "left_knee_angle": left_knee_angle,
        "right_knee_angle": right_knee_angle,
        "left_waist_angle": left_waist_angle,
        "right_waist_angle": right_waist_angle
    }

In [8]:
pose_classification = classify_pose(detection_result.pose_landmarks[0])

In [10]:
cv2.putText(annotated_image, pose_classification["label"], (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
cv2.putText(annotated_image, f"Left Knee Angle: {pose_classification['left_knee_angle']:.2f}", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
cv2.putText(annotated_image, f"Right Knee Angle: {pose_classification['right_knee_angle']:.2f}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
cv2.putText(annotated_image, f"Left Waist Angle: {pose_classification['left_waist_angle']:.2f}", (50, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
cv2.putText(annotated_image, f"Right Waist Angle: {pose_classification['right_waist_angle']:.2f}", (50, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)


cv2.imshow("Show image", cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()